# Add Features to the `Summer_Athlete` Dataset

In [85]:
import pandas as pd
athlete = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\Clean_Athlete_df.csv')
medal_host = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\Clean_Medal_Host_df.csv')
#**Medal host has data from year 2000 to 2024 but we want to save 2024 data for testing!!**

In [86]:
# Add number of sports an athlete plays to the df .transform(method.. 'nunique')
athlete['Number of Sports Athlete Plays'] = athlete.groupby('Name')['Sport'].transform('nunique')

# Add number of events an athlete plays to the df .transform(method.. 'nunique')
athlete['Number of Events Athlete Competes In'] = athlete.groupby('Name')['Event'].transform('nunique')

# Add number of years an athlete participates in the olympics
athlete['Number of Games Athlete Competed In'] = athlete.groupby('Name')['Year'].transform('nunique')

# Create Boolean Columns for Each Medal Type
athlete['Won_Gold_True'] = (athlete['Medal'] == 'Gold').astype(int)
athlete['Won_Silver_True'] = (athlete['Medal'] == 'Silver').astype(int)
athlete['Won_Bronze_True'] = (athlete['Medal'] == 'Bronze').astype(int)

#Number of each medal type the athlete won OVER ALL YEARS:
athlete["Athlete's Gold Medals"] = 0
athlete["Athlete's Gold Medals"] = athlete.groupby(['Name'])['Medal'].transform(lambda x: (x == 'Gold').sum())
athlete["Athlete's Silver Medals"] = 0
athlete["Athlete's Silver Medals"] = athlete.groupby(['Name'])['Medal'].transform(lambda x: (x == 'Silver').sum())
athlete["Athlete's Bronze Medals"] = 0
athlete["Athlete's Bronze Medals"] = athlete.groupby(['Name'])['Medal'].transform(lambda x: (x == 'Bronze').sum())

# Dummy column for gender
athlete['Male_True'] = (athlete['Sex'] == 'M').astype(int)
athlete.drop(['Sex', 'City'], axis=1, inplace=True)

athlete.drop(columns=['Event', 'Team', 'Medal','Sport'], axis =1, inplace=True)
athlete.drop_duplicates(subset=['Name', 'NOC', 'Year'], inplace=True)

In [87]:
#Output
Cleaned_Athlete_W_New_Features = athlete.copy()
Cleaned_Athlete_W_New_Features.to_csv('Athlete_Dataset_New_Features.csv', index = False)
Cleaned_Athlete_W_New_Features.head()

,Name,NOC,Year,Number of Sports Athlete Plays,Number of Events Athlete Competes In,Number of Games Athlete Competed In,Won_Gold_True,Won_Silver_True,Won_Bronze_True,Athlete's Gold Medals,Athlete's Silver Medals,Athlete's Bronze Medals,Male_True
0,A Lamusi,CHN,2012,1,1,1,0,0,0,0,0,0,1
1,Jyri Aalto,FIN,2000,1,1,1,0,0,0,0,0,0,1
2,Minna Aalto,FIN,2000,1,1,1,0,0,0,0,0,0,0
3,Timo Aaltonen,FIN,2000,1,1,1,0,0,0,0,0,0,1
4,Ragnhild Aamodt,NOR,2008,1,1,1,1,0,0,1,0,0,0


# Transform the `Summer_Athlete` DF into `NOC_Team_Data`

**Creating a Dataset with Nation's Teams Medal information for each year between 2000-2020**

In [88]:
# Create 2024 NOC stats DataFrame
athlete_copy = athlete.copy()
NOC_Team_Data = (athlete_copy.groupby(['NOC', 'Year']).agg(
                                Total_Athletes_in_Team=('Name', 'nunique'),  # Counting unique athletes
                                Gold_Winning_Athletes=('Won_Gold_True', lambda x: (x > 0).sum()),  # Sum of gold-winning athletes
                                Silver_Winning_Athletes=('Won_Silver_True', lambda x: (x > 0).sum()),  # Sum of silver-winning athletes
                                Bronze_Winning_Athletes=('Won_Bronze_True', lambda x: (x > 0).sum()),  # Sum of bronze-winning athletes
                                Athletes_Who_Returned=('Number of Games Athlete Competed In', lambda x: (x > 1).sum()),  # Count of athletes who returned
                                Multi_Event_Athletes = ('Number of Events Athlete Competes In', lambda x: (x > 1).sum()),)
                                .reset_index()  # Reset index to turn groupby variables into columns
)

# Preview the resulting DataFrame
NOC_Team_Data.head()

,NOC,Year,Total_Athletes_in_Team,Gold_Winning_Athletes,Silver_Winning_Athletes,Bronze_Winning_Athletes,Athletes_Who_Returned,Multi_Event_Athletes
0,AFG,2004,5,0,0,0,2,0
1,AFG,2008,4,0,0,1,4,2
2,AFG,2012,6,0,0,1,3,2
3,AFG,2016,3,0,0,0,0,0
4,AFG,2020,5,0,0,0,0,0


# Create **Team-Based** feature to ratings nation's teams based on `Cleaned_Athlete_Dataset_New_Features` 

In [89]:
import math

In [90]:
# Define the function to calculate team rating
def calculate_team_rating(df):
    # Calculate the athlete score for each row
    df['Athlete Score'] = (math.log(10)) * (
        df["Athlete's Gold Medals"] +
        df["Athlete's Silver Medals"] +
        df["Athlete's Bronze Medals"] +
        df['Number of Games Athlete Competed In']
    )
    # Group by NOC and Year, calculate the average score for each group
    result = df.groupby(['NOC', 'Year'], as_index=False)['Athlete Score'].mean()
    
    # Rename the score column for clarity
    result = result.rename(columns={'Athlete Score': 'Average Team Rating'})
    
    return result

In [91]:
#Making Team Rating for each Nation's Team:
athlete_feature_df = Cleaned_Athlete_W_New_Features.copy()
team_rating = calculate_team_rating(athlete)
team_rating.head()

,NOC,Year,Average Team Rating
0,AFG,2004,3.684136
1,AFG,2008,6.332109
2,AFG,2012,4.605170
3,AFG,2016,2.302585
4,AFG,2020,2.302585


# Merging Nation's Team Data and exporting it. 

In [92]:
# Perform the join on 'nation_name' and 'year'
NOC_Data = NOC_Team_Data.merge(team_rating, on = ['NOC', 'Year'], how='left')
NOC_Data.head()

,NOC,Year,Total_Athletes_in_Team,Gold_Winning_Athletes,Silver_Winning_Athletes,Bronze_Winning_Athletes,Athletes_Who_Returned,Multi_Event_Athletes,Average Team Rating
0,AFG,2004,5,0,0,0,2,0,3.684136
1,AFG,2008,4,0,0,1,4,2,6.332109
2,AFG,2012,6,0,0,1,3,2,4.605170
3,AFG,2016,3,0,0,0,0,0,2.302585
4,AFG,2020,5,0,0,0,0,0,2.302585


In [93]:
# Adding Medal Counts and Hosting information from Cleaned_Medal_Host...
#**Medal host has data from year 2000 to 2024 and we want to save 2024 data for testing!! This appends the 2024 to the NOC Team data but leaves the team data null for 2024**
NOC_Data = pd.merge(medal_host, NOC_Data, on=['NOC', 'Year'], how='left')
NOC_Data.drop(columns=['Hosting'], axis=1, inplace=True)

#check that 2024 prediction data is null while testing data is full for 2024
NOC_Data_2024 = NOC_Data[NOC_Data['Year'] == 2024]
NOC_Data_2024.head(10)

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Total_Athletes_in_Team,Gold_Winning_Athletes,Silver_Winning_Athletes,Bronze_Winning_Athletes,Athletes_Who_Returned,Multi_Event_Athletes,Average Team Rating
465,1,USA,40,44,42,126,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,2,CHN,40,27,24,91,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,3,JPN,20,12,13,45,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,4,AUS,18,19,16,53,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,5,FRA,16,26,22,64,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,6,NLD,15,7,12,34,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,7,GBR,14,22,29,65,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,9,ITA,12,13,15,40,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,10,DEU,12,13,8,33,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
474,11,NZL,10,7,3,20,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
#Output
NOC_Data.to_csv("Nation's Team Data by Year.csv", index = False)